In [1]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import anndata as an
from kneed import KneeLocator

### 1. Prepare TCDG files

In [2]:
pert_type = "crispra"  # or "crispri"

In [3]:
df = pd.read_csv(f'/home/user/Documents/Kinase_project/figures/decipher_analysis/NTC_{pert_type}/T_cell_dose_gene.csv')
T_cell_dose_gene = df['gene_short_name']

In [4]:
T_cell_dose_gene

0           TSPAN6
1              CFH
2             GCLC
3           NIPAL3
4              BAD
           ...    
4018          CDR1
4019    FP565324.1
4020    AC116366.3
4021    AC004922.1
4022    AC119674.2
Name: gene_short_name, Length: 4023, dtype: object

### 2. Generate raw and hvg files

In [5]:
gbm= an.read_h5ad(f'/home/user/Documents/Kinase_project/{pert_type}_final/preprocessed_cds_without_Tcells.h5ad')

In [6]:
gbm

AnnData object with n_obs × n_vars = 313199 × 56648
    obs: 'P7', 'P5', 'sample', 'n.umi', 'log10.umi', 'percent_mito', 'cell_ID', 'RT', 'Lig', 'new_cell', 'oligo', 'total_reads.x', 'total_hash_umis_per_cell_ID', 'top_to_second_best_ratio', 'treatment', 'sgRNA', 'total_reads.y', 'sgRNA_proportion', 'total_sgrna_read_per_cell', 'rank', 'top_to_second', 'second_to_third', 'third_to_next', 'top_proportion', 'second_proportion', 'third_proportion', 'top_sg', 'second_sg', 'third_sg', 'gene', 'second_sg_gene', 'third_sg_gene', 'cell', 'RT_lig', 'num_genes_expressed', 'Size_Factor', 'gene_id', 'replicate', 'g1s_score', 'g2m_score', 'proliferation_index', 'dose', 'UMAP1', 'UMAP2', 'PCA_Cluster'
    var: 'features'

In [7]:
# Filter out cells where sgRNA is NA
gbm = gbm[~gbm.obs['sgRNA'].isna()].copy()
gbm

AnnData object with n_obs × n_vars = 313199 × 56648
    obs: 'P7', 'P5', 'sample', 'n.umi', 'log10.umi', 'percent_mito', 'cell_ID', 'RT', 'Lig', 'new_cell', 'oligo', 'total_reads.x', 'total_hash_umis_per_cell_ID', 'top_to_second_best_ratio', 'treatment', 'sgRNA', 'total_reads.y', 'sgRNA_proportion', 'total_sgrna_read_per_cell', 'rank', 'top_to_second', 'second_to_third', 'third_to_next', 'top_proportion', 'second_proportion', 'third_proportion', 'top_sg', 'second_sg', 'third_sg', 'gene', 'second_sg_gene', 'third_sg_gene', 'cell', 'RT_lig', 'num_genes_expressed', 'Size_Factor', 'gene_id', 'replicate', 'g1s_score', 'g2m_score', 'proliferation_index', 'dose', 'UMAP1', 'UMAP2', 'PCA_Cluster'
    var: 'features'

In [8]:
gbm = gbm[~gbm.obs['dose'].isna()].copy()
gbm

AnnData object with n_obs × n_vars = 313199 × 56648
    obs: 'P7', 'P5', 'sample', 'n.umi', 'log10.umi', 'percent_mito', 'cell_ID', 'RT', 'Lig', 'new_cell', 'oligo', 'total_reads.x', 'total_hash_umis_per_cell_ID', 'top_to_second_best_ratio', 'treatment', 'sgRNA', 'total_reads.y', 'sgRNA_proportion', 'total_sgrna_read_per_cell', 'rank', 'top_to_second', 'second_to_third', 'third_to_next', 'top_proportion', 'second_proportion', 'third_proportion', 'top_sg', 'second_sg', 'third_sg', 'gene', 'second_sg_gene', 'third_sg_gene', 'cell', 'RT_lig', 'num_genes_expressed', 'Size_Factor', 'gene_id', 'replicate', 'g1s_score', 'g2m_score', 'proliferation_index', 'dose', 'UMAP1', 'UMAP2', 'PCA_Cluster'
    var: 'features'

In [9]:
gbm.X.max()

4161.0

In [10]:
gbm_crop = gbm[
    (gbm.obs['total_sgrna_read_per_cell'] > 3) & 
    (gbm.obs['top_proportion'] > 0.3)
].copy()
gbm_crop

AnnData object with n_obs × n_vars = 255701 × 56648
    obs: 'P7', 'P5', 'sample', 'n.umi', 'log10.umi', 'percent_mito', 'cell_ID', 'RT', 'Lig', 'new_cell', 'oligo', 'total_reads.x', 'total_hash_umis_per_cell_ID', 'top_to_second_best_ratio', 'treatment', 'sgRNA', 'total_reads.y', 'sgRNA_proportion', 'total_sgrna_read_per_cell', 'rank', 'top_to_second', 'second_to_third', 'third_to_next', 'top_proportion', 'second_proportion', 'third_proportion', 'top_sg', 'second_sg', 'third_sg', 'gene', 'second_sg_gene', 'third_sg_gene', 'cell', 'RT_lig', 'num_genes_expressed', 'Size_Factor', 'gene_id', 'replicate', 'g1s_score', 'g2m_score', 'proliferation_index', 'dose', 'UMAP1', 'UMAP2', 'PCA_Cluster'
    var: 'features'

In [11]:
# Subset the AnnData object using the common genes
gbm_crop_tcdg = gbm_crop[:, gbm_crop.var['features'].isin(T_cell_dose_gene.values)]
gbm_crop_tcdg.var.rename(columns={'_index':'index'},inplace=True)
gbm_crop_tcdg.raw.var.rename(columns={'_index':'index'},inplace=True) 
gbm_crop_tcdg.write_h5ad(f'/home/user/Documents/Kinase_project/{pert_type}_final/gbm_{pert_type}_crop_tcdg.h5ad')

/home/user/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/user/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/user/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/user/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/user/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/user/anaconda3